In [1]:
face_url = "centralindia.cognitiveservices.azure.com"
face_key = "5d16a60a502840af8425701f46456ecd"

In [5]:
import json
import urllib.parse 
import http.client 

persongroup_headers = {
    'Ocp-Apim-Subscription-Key':face_key,
    'Content-Type':'application/json'
}
persongroup_body = {
    'name':'FriendsList'
}
persongroup_id = "neighbours"
try:
    pgconn = http.client.HTTPSConnection(face_url)
    pgconn.request('PUT', '/face/v1.0/persongroups/%s' % persongroup_id, str(persongroup_body), persongroup_headers)
    response = pgconn.getresponse()
    print(response.read())
    pgconn.close()
except Exception as e:
    print(e)

b'{"error":{"code":"PersonGroupExists","message":"Person group \'neighbours\' already exists."}}'


In [8]:
person_body = {
    'name':'Jordan'
}

person_url = '/face/v1.0/persongroups/'+persongroup_id+'/persons'

try:
    pconn = http.client.HTTPSConnection(face_url)
    pconn.request('POST', person_url, str(person_body), persongroup_headers)
    response = pconn.getresponse()
    data = json.loads(response.read())
    personid=data["personId"]
    print(personid)
    pconn.close()
except Exception as e:
    print(e)

2ab2f399-27c4-47db-9fd6-3df5041fce61


In [13]:
myneighbourlist = ["http://media1.s-nbcnews.com/i/newscms/2014_19/425336/140507-jordan-0419_9c7ff3cf5b7db48241b72793c341e64a.jpg",
                "https://ladyinthemancave.files.wordpress.com/2014/07/cave33.jpg",
                "https://cdn-s3.si.com/s3fs-public/images/1997-Michael-Jordan-05717484.jpg",
                "https://statics.sportskeeda.com/editor/2018/03/a4a7b-1520474015-800.jpg",
                "https://thefederalist.com/wp-content/uploads/2016/07/chi-michael-jordan-photos-008.jpeg",
                "https://servingjoy.com/wp-content/uploads/2015/01/American-basketball-legend-Michael-Jordan-at-a-Special-Dinner-hosted-by-him-at-the-Roundhouse-in-October-2006.jpg"]

face_endpoint = '/face/v1.0/persongroups/'+persongroup_id+'/persons/'+personid+'/persistedFaces' 

for image in myneighbourlist:
    face_body = {
        'url' : image
    }
    try:
        fconn = http.client.HTTPSConnection(face_url)
        fconn.request('POST', face_endpoint, str(face_body), persongroup_headers)
        response = fconn.getresponse()
        data = json.loads(response.read())
        faceid=data["persistedFaceId"]
        print(faceid)
        fconn.close()
    except Exception as e:
        print(e)

1d8e4035-3ff6-49c3-aa37-aa24e1fe1789
aca48a6a-fdd4-47c6-9e4b-55603caddf63
8cd010f5-18ce-4851-acb1-e8e32ac92e51
f06835a8-18c5-4283-9531-f8339a4f936a
091c112b-83df-4b8d-bbd3-7fca191e5946
72655963-d16e-403d-afae-d0fd0ed7be32


In [14]:
train_faceendpoint = '/face/v1.0/persongroups/'+persongroup_id+'/train'

try:
    tconn = http.client.HTTPSConnection(face_url)
    tconn.request('POST', train_faceendpoint, '', persongroup_headers)
    response = tconn.getresponse()
    data = response.read()
    print(data)
    tconn.close()
except Exception as e:
    print(e)

b''


In [49]:
detect_body = {
    'url':'http://media.gq.com/photos/57a235c218941d5256600e00/16:9/pass/michael-jordan-camp.jpg'
}
detect_params = urllib.parse.urlencode({
    'returnFaceId':True
})

detect_endpoint = '/face/v1.0/detect?%s'

try:
    dconn = http.client.HTTPSConnection(face_url)
    dconn.request('POST', detect_endpoint % detect_params,str(detect_body), persongroup_headers)
    response = dconn.getresponse()
    data = json.loads(response.read())
    detect_faceid = data[0]['faceId']
    print(detect_faceid)
    dconn.close()
except Exception as ex:
    print(ex)

dcfa2a6b-d41a-420f-b72f-b5dd2469d7a8


In [50]:
identify_body = {
    'faceIds':[detect_faceid],
    'personGroupId':persongroup_id
}

identify_endpoint = '/face/v1.0/identify'

try:
    iconn = http.client.HTTPSConnection(face_url)
    iconn.request('POST', identify_endpoint, str(identify_body), persongroup_headers)
    response = iconn.getresponse()
    data = json.loads(response.read())
    identifiedcandidate=data[0]["candidates"]
    if identifiedcandidate == []:
        print("The person is not matching with your neighbours list")
    elif identifiedcandidate[0]["confidence"] > 0.5:
        identifiedperson = identifiedcandidate[0]["personId"]
        print(identifiedperson)
    iconn.close()
except Exception as ex:
    print(ex)

2ab2f399-27c4-47db-9fd6-3df5041fce61


In [41]:
getname_endpoint = '/face/v1.0/persongroups/'+persongroup_id+'/persons/'+identifiedperson

try:
    getconn = http.client.HTTPSConnection(face_url)
    getconn.request('GET', getname_endpoint, '', persongroup_headers)
    response = getconn.getresponse()
    data = json.loads(response.read())    
    print("The identified person is "+str(data["name"]))
    getconn.close()
except Exception as e:
    print(e)

The identified person is Jordan
